## Imports

In [ ]:
import json
import pandas as pd
import altair as alt
import itables as itables
import ipywidgets as widgets
import plotly.offline as plotly
import plotly.express as px

plotly.init_notebook_mode(connected=True)  
itables.init_notebook_mode(all_interactive=True)

## Load data

In [ ]:
# cardano_node_0_df, cardano_node_0_df and all_df
def df_from_cardano_node_logs(fp):
    lines = open(fp, "r").readlines()
    at_lines = [ json.loads(line) for line in lines if line.startswith('{"at') ]
    return pd.DataFrame.from_records(at_lines)


def events_chart(df, y, color):
    fig = px.scatter(
        df, 
        x='at', 
        y=y, 
        color=color, # Optional: Group events by color
        title='Event chart',
        hover_data=['data'] # Show event name on hover
    )

    # Add the Range Slider and Range Selector Buttons
    fig.update_xaxes(
        # Enable the Range Slider below the chart
        rangeslider_visible=True,
        
        # Add preset buttons for easy selection (e.g., 1 month, 6 months)
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1M", step="month", stepmode="backward"),
                dict(count=6, label="6M", step="month", stepmode="backward"),
                dict(step="all", label="ALL")
            ])
        )
     )
    return fig

cardano_node_0_df = df_from_cardano_node_logs("data/cardano-node-0.log")
cardano_node_1_df = df_from_cardano_node_logs("data/cardano-node-1.log")

all_df = pd.concat([cardano_node_0_df.assign(source="cardano-node-0"), cardano_node_1_df.assign(source="cardano-node-1")])

all_df

## Quick refresher

In [ ]:
# Events

events_chart(all_df, 'source', 'ns')

In [ ]:
# SELECT * FROM all_df WHERE sev = 'Error'
display(events_chart(all_df[all_df.sev == "Error"], "source", "ns"))
all_df[all_df.sev == "Error"]

In [ ]:
# SELECT DISTINCT_COUNT(ns), ARG_MAX(ns), ARG_MIN(ns), ARG_MAX(sev), ARG_MIN(sev) FROM all_df GROUP BY source 
all_df.groupby('source').agg({
    'ns': ['nunique', 'max', 'min'],
    'sev': ['max', 'min']
})

## Block related logs

In [ ]:
block_df = all_df[all_df.data.apply(lambda r: 'block' in r)]

events_chart(block_df, "source", "ns")